In [1]:
import os, re, unicodedata
from datetime import datetime
from pyspark.sql import SparkSession, functions as F
from notebookutils import mssparkutils

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 3, Finished, Available, Finished)

In [2]:
# ---------- Spark ----------
spark = SparkSession.builder.getOrCreate()

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 4, Finished, Available, Finished)

In [3]:
# Seleccionar el Lakehouse destino (silver)
spark.sql("USE lh_silver")

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 5, Finished, Available, Finished)

DataFrame[]

In [4]:
# 📂 Carpetas bronze (donde ya están los JSON)
bronze_base_logical = "Files/bronze/ESIOS/data/balance/balance-electrico"
bronze_base_physical = "/lakehouse/default/Files/bronze/ESIOS/data/balance/balance-electrico"

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 6, Finished, Available, Finished)

In [5]:
# 📂 Salida (Tables/)
tables_prefix = "Tables"

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 7, Finished, Available, Finished)

In [6]:
# --------------------------
# Función utilitaria mejorada
# --------------------------
def slugify(name: str) -> str:
    # Normalizar y quitar tildes
    name = unicodedata.normalize("NFKD", name)
    name = name.encode("ascii", "ignore").decode("utf-8")  # quita acentos
    # Sustituir caracteres no válidos por "_"
    return re.sub(r'[^a-z0-9_]', '_', name.lower())

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 8, Finished, Available, Finished)

In [7]:
# 📊 DEFINICIÓN DE CATEGORÍAS DE ENERGÍA
CATEGORIAS_ENERGIA = {
    "renovables": [
        "hidraulica",
        "eolica", 
        "solar_fotovoltaica",
        "solar_termico",
        "otras_renovables",
        "turbinacion_bombeo"
    ],
    "no_renovables": [
        "ciclo_combinado",
        "carbon",
        "cogeneracion",
        "residuos_no_renovables"
    ],
    "sistema": [
        "consumo_de_bombeo",
        "saldo_intercambios"
    ]
}

print("✅ Categorías definidas correctamente")
for categoria, tipos in CATEGORIAS_ENERGIA.items():
    print(f"   {categoria}: {len(tipos)} tipos de energía")

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 9, Finished, Available, Finished)

✅ Categorías definidas correctamente
   renovables: 6 tipos de energía
   no_renovables: 4 tipos de energía
   sistema: 2 tipos de energía


In [8]:
# 🚀 FUNCIÓN PARA CREAR TABLAS AGREGADAS
def crear_tabla_agregada(categoria_nombre, tipos_energia):
    """
    Crea una tabla agregada para una categoría específica
    """
    print(f"\n📊 Creando tabla agregada: {categoria_nombre}")
    
    # Lista para almacenar todos los DataFrames de la categoría
    dfs_categoria = []
    
    for tipo_energia in tipos_energia:
        try:
            # Leer la tabla Delta existente
            table_name = f"slv_esios_balance_{tipo_energia}_month"
            df = spark.read.format("delta").table(table_name)
            
            # Agregar columna de tipo de energía
            df = df.withColumn("tipo_energia", F.lit(tipo_energia))
            dfs_categoria.append(df)
            
            print(f"   ✅ Añadido: {tipo_energia} ({df.count()} registros)")
            
        except Exception as e:
            print(f"   ⚠️  No se pudo leer {tipo_energia}: {e}")
    
    if dfs_categoria:
        # Unir todos los DataFrames de la categoría
        df_agregado = dfs_categoria[0]
        for df in dfs_categoria[1:]:
            df_agregado = df_agregado.union(df)
        
        # Guardar tabla agregada
        table_name = f"slv_esios_balance_agregado_{categoria_nombre}_month"
        df_agregado.write.format("delta").mode("overwrite").saveAsTable(table_name)
        
        print(f"   💾 Guardada tabla: {table_name} ({df_agregado.count()} registros totales)")
        return df_agregado
    else:
        print(f"   ❌ No se pudieron leer datos para la categoría {categoria_nombre}")
        return None

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 10, Finished, Available, Finished)

In [9]:
# 📈 FUNCIÓN PARA CREAR TABLA DE GENERACIÓN TOTAL
def crear_tabla_total_generacion():
    """
    Crea tabla con la generación total del sistema
    """
    print(f"\n📈 Creando tabla de generación total del sistema")
    
    # Leer todas las tablas de generación (excluyendo consumo e intercambios)
    tipos_generacion = CATEGORIAS_ENERGIA["renovables"] + CATEGORIAS_ENERGIA["no_renovables"]
    
    dfs_generacion = []
    
    for tipo_energia in tipos_generacion:
        try:
            table_name = f"slv_esios_balance_{tipo_energia}_month"
            df = spark.read.format("delta").table(table_name)
            df = df.withColumn("tipo_energia", F.lit(tipo_energia))
            dfs_generacion.append(df)
            print(f"   ✅ Añadido: {tipo_energia}")
            
        except Exception as e:
            print(f"   ⚠️  No se pudo leer {tipo_energia}: {e}")
    
    if dfs_generacion:
        # Unir todos los DataFrames
        df_total = dfs_generacion[0]
        for df in dfs_generacion[1:]:
            df_total = df_total.union(df)
        
        # Calcular total por fecha y comunidad
        df_total_agregado = (df_total
            .groupBy("datetime", "geo_id", "geo_name", "year")
            .agg(F.sum("value").alias("generacion_total"))
            .withColumn("tipo_energia", F.lit("total_generacion"))
        )
        
        # Guardar tabla
        table_name = "slv_esios_balance_total_generacion_month"
        df_total_agregado.write.format("delta").mode("overwrite").saveAsTable(table_name)
        
        print(f"   💾 Guardada tabla: {table_name} ({df_total_agregado.count()} registros)")
        return df_total_agregado
    
    return None

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 11, Finished, Available, Finished)

In [10]:
# 🌍 FUNCIÓN PARA CREAR TABLA DE MIX ENERGÉTICO
def crear_tabla_mix_energetico():
    """
    Crea tabla con el mix energético (distribución porcentual por tipo)
    """
    print(f"\n🌍 Creando tabla de mix energético")
    
    try:
        # Leer tabla de generación total
        df_total = spark.read.format("delta").table("slv_esios_balance_total_generacion_month")
        
        # Leer todas las tablas individuales
        tipos_generacion = CATEGORIAS_ENERGIA["renovables"] + CATEGORIAS_ENERGIA["no_renovables"]
        
        dfs_mix = []
        
        for tipo_energia in tipos_generacion:
            try:
                table_name = f"slv_esios_balance_{tipo_energia}_month"
                df_tipo = spark.read.format("delta").table(table_name)
                
                # Unir con total para calcular porcentaje
                df_mix_tipo = (df_tipo
                    .join(df_total, ["datetime", "geo_id", "geo_name", "year"])
                    .withColumn("porcentaje", F.col("value") / F.col("generacion_total") * 100)
                    .withColumn("tipo_energia", F.lit(tipo_energia))
                    .select("datetime", "geo_id", "geo_name", "year", "tipo_energia", 
                           "value", "generacion_total", "porcentaje")
                )
                
                dfs_mix.append(df_mix_tipo)
                print(f"   ✅ Calculado mix para: {tipo_energia}")
                
            except Exception as e:
                print(f"   ⚠️  Error calculando mix para {tipo_energia}: {e}")
        
        if dfs_mix:
            # Unir todos los DataFrames del mix
            df_mix_completo = dfs_mix[0]
            for df in dfs_mix[1:]:
                df_mix_completo = df_mix_completo.union(df)
            
            # Guardar tabla
            table_name = "slv_esios_balance_mix_energetico_month"
            df_mix_completo.write.format("delta").mode("overwrite").saveAsTable(table_name)
            
            print(f"   💾 Guardada tabla: {table_name} ({df_mix_completo.count()} registros)")
            return df_mix_completo
            
    except Exception as e:
        print(f"   ❌ Error creando mix energético: {e}")
    
    return None

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 12, Finished, Available, Finished)

# 🚀 EJECUCIÓN PRINCIPAL

A continuación se ejecuta el proceso completo de creación de tablas agregadas:

In [11]:
print('=' * 60)
print("🚀 INICIANDO CREACIÓN DE TABLAS AGREGADAS POR CATEGORÍA")
print('=' * 60)

# 1. Tablas agregadas por categoría
tabla_renovables = crear_tabla_agregada("renovables", CATEGORIAS_ENERGIA["renovables"])
tabla_no_renovables = crear_tabla_agregada("no_renovables", CATEGORIAS_ENERGIA["no_renovables"])
tabla_sistema = crear_tabla_agregada("sistema", CATEGORIAS_ENERGIA["sistema"])

# 2. Tabla de generación total
tabla_total = crear_tabla_total_generacion()

# 3. Tabla de mix energético (solo si tenemos generación total)
if tabla_total is not None:
    tabla_mix = crear_tabla_mix_energetico()

print("\n")
print('=' * 60)
print("✅ PROCESO COMPLETADO")
print('=' * 60)

# Mostrar resumen de tablas creadas
print("\n📋 TABLAS CREADAS EN EL LAKEHOUSE SILVER:")
print("   - slv_esios_balance_agregado_renovables_month")
print("   - slv_esios_balance_agregado_no_renovables_month") 
print("   - slv_esios_balance_agregado_sistema_month")
print("   - slv_esios_balance_total_generacion_month")
print("   - slv_esios_balance_mix_energetico_month")

print("\n🎯 USO DE LAS TABLAS:")
print("   • Análisis de tendencias por categoría")
print("   • Comparativas renovables vs no renovables")
print("   • Evolución del mix energético")
print("   • Distribución porcentual por tecnología")

print(f"\n🕒 Proceso terminado - {datetime.utcnow().isoformat()}Z")

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 13, Finished, Available, Finished)

🚀 INICIANDO CREACIÓN DE TABLAS AGREGADAS POR CATEGORÍA

📊 Creando tabla agregada: renovables
   ✅ Añadido: hidraulica (446 registros)
   ✅ Añadido: eolica (430 registros)
   ✅ Añadido: solar_fotovoltaica (515 registros)
   ✅ Añadido: solar_termico (168 registros)
   ✅ Añadido: otras_renovables (476 registros)
   ✅ Añadido: turbinacion_bombeo (280 registros)
   💾 Guardada tabla: slv_esios_balance_agregado_renovables_month (2315 registros totales)

📊 Creando tabla agregada: no_renovables
   ✅ Añadido: ciclo_combinado (363 registros)
   ✅ Añadido: carbon (60 registros)
   ✅ Añadido: cogeneracion (448 registros)
   ✅ Añadido: residuos_no_renovables (306 registros)
   💾 Guardada tabla: slv_esios_balance_agregado_no_renovables_month (1177 registros totales)

📊 Creando tabla agregada: sistema
   ✅ Añadido: consumo_de_bombeo (280 registros)
   ✅ Añadido: saldo_intercambios (448 registros)
   💾 Guardada tabla: slv_esios_balance_agregado_sistema_month (728 registros totales)

📈 Creando tabla de 

# 📊 VERIFICACIÓN DE TABLAS CREADAS

Para verificar que las tablas se han creado correctamente, podemos listarlas:

In [12]:
# Listar tablas creadas en el lakehouse silver
print("📋 TABLAS DISPONIBLES EN EL LAKEHOUSE SILVER:")
tablas = spark.sql("SHOW TABLES")
tablas_agregadas = tablas.filter(F.col("tableName").like("%agregado%") | F.col("tableName").like("%total%") | F.col("tableName").like("%mix%"))
tablas_agregadas.show(truncate=False)

StatementMeta(, b1b09caf-1889-4d36-bda6-b7172b882ea8, 14, Finished, Available, Finished)

📋 TABLAS DISPONIBLES EN EL LAKEHOUSE SILVER:
+---------+----------------------------------------------+-----------+
|namespace|tableName                                     |isTemporary|
+---------+----------------------------------------------+-----------+
|lh_silver|slv_esios_balance_agregado_renovables_month   |false      |
|lh_silver|slv_esios_balance_agregado_no_renovables_month|false      |
|lh_silver|slv_esios_balance_agregado_sistema_month      |false      |
|lh_silver|slv_esios_balance_total_generacion_month      |false      |
|lh_silver|slv_esios_balance_mix_energetico_month        |false      |
+---------+----------------------------------------------+-----------+

